In [58]:
%load_ext autoreload
%autoreload 2

# import libraries
import numpy as np
import matplotlib.pyplot as plt
import pyshtools as pysh
import SLmod as SL
from numpy import pi as pi

#set plotting parameters
plt.rc('xtick', labelsize=18) 
plt.rc('ytick', labelsize=18) 
font = {'size'   : 20}
plt.rc('font', **font)
plt.rcParams['figure.figsize'] = [16, 8]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
def psi_lm_calc(om,L = 2):
    psi = pysh.SHGrid.from_zeros(lmax=L)
    ilat = 0
    for lat in psi.lats():
        th = (90-lat)*pi/180
        fac = np.cos(th)*np.sin(th)
        ilon = 0
        for lon in psi.lons():
            ph = lon*pi/180
            psi.data[ilat,ilon] = fac*(om[0]*np.cos(ph) + om[1]*np.sin(ph))
            ilon += 1
        ilat += 1
    psi_lm = psi.expand(lmax_calc = 2,normalization = 'ortho')
    return psi_lm

om = np.zeros(2)
om[1] = 1
psi_lm = psi_lm_calc(om)*np.sqrt(15/(4*pi))
print(psi_lm.coeffs[0,2,1])
print(psi_lm.coeffs[1,2,1])
#psi_2m = SL.centrifugal_perturbation(om)
#print(psi_2m[0,1])
#print(psi_2m[1,1])


-9.57148215874225e-17
0.9999999999999999


In [16]:
def interia_calc(L = 2):
    f = pysh.SHGrid.from_zeros(lmax=L,kind = 'complex')
    ilat = 0
    for lat in f.lats():
        th = (90-lat)*pi/180
        ilon = 0
        for lon in f.lons():
            ph = lon*pi/180
            f.data[ilat,ilon] = np.cos(th)*np.sin(th)*np.cos(ph)
            ilon += 1
        ilat += 1
    f_lm = f.expand(lmax_calc = 2,normalization = 'ortho',csphase = -1)
    return f_lm.coeffs[:,2,:]

f_lm = interia_calc()
print(f_lm)


[[ 9.16703587e-19+0.00000000e+00j -6.47208638e-01+9.81826947e-17j
  -1.68409203e-18+6.73636810e-18j]
 [ 0.00000000e+00+0.00000000e+00j  6.47208638e-01+9.81826947e-17j
  -1.68409203e-18-6.73636810e-18j]]
